# Nodding Detector

This notebook builds upon the notebook "Action detection for sign language" by Nicholas Renotte and adapts it to the usecase of detecting nodding gestures:
https://github.com/nicknochnack/ActionDetectionforSignLanguage/blob/main/Action%20Detection%20Tutorial.ipynb

In [ ]:
!conda env list
!python --version

In [ ]:
!pip install mediapipe tensorflow==2.4.1 opencv-python sklearn matplotlib

In [ ]:
!pip install scikit-learn #sklearn vs. scikit-learn, see here: https://towardsdatascience.com/scikit-learn-vs-sklearn-6944b9dc1736

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [ ]:
mp_facemesh = mp.solutions.face_mesh # Only Facemesh instead of Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [ ]:
def draw_styled_landmarks(image, results):
    if not results:
        return
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.multi_face_landmarks[0], mp_facemesh.FACEMESH_TESSELATION) 

## 1. Testing the capturing of facial landmarks

Note: On Mac, the cv2.destroyAllWindows() doesn't work. The window stays open and freezes

In [ ]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_facemesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5, static_image_mode=True, max_num_faces=1, refine_landmarks=False,) as face:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, face)
        if results and results.multi_face_landmarks:    
            print(results.multi_face_landmarks[0].landmark[0].x)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    #cv2.destroyAllWindows()

In [ ]:
last_frame_keypoints = results.multi_face_landmarks[0].landmark
len(last_frame_keypoints) # number of keypoints detected in one frame

## 2. Extract keypoint values

In [ ]:
ref_pt = singleresult[1]
aligned_landmarks = [{"x": pt.x - ref_pt.x, "y": pt.y - ref_pt.y, "z": pt.z - ref_pt.z} for pt in singleresult]
aligned_landmarks

In [ ]:
#TODO
def normalize_landmarks(landmarks, reference_size=256):
    #align_and_scale
    ref_pt = landmarks[1] #Nosetip
    aligned_landmarks = [{"x": pt.x - ref_pt.x, "y": pt.y - ref_pt.y, "z": pt.z - ref_pt.z} for pt in landmarks]
    scale_factor = reference_size / np.max(np.abs(aligned_landmarks))
    scaled_landmarks = aligned_landmarks * scale_factor
    return scaled_landmarks

In [ ]:
def extract_keypoints(results):
    #TODO: normalize first
    NOSE_INDEX = 1
    relevant_points = [NOSE_INDEX, 2, 3] #TODO: Select more expressive points
    data = [0, 0, 0] * len(relevant_points) #fallback if no data
    if results.multi_face_landmarks[0].landmark:
        data = np.array([[res.x, res.y, res.z] for res in np.array(results.multi_face_landmarks[0].landmark)[[relevant_points]]]).flatten()
    return data

## 3. Setup folders to store training data

In [ ]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
ACTIONS = np.array(['nodding', 'naying', 'random'])

# Thirty videos per action
N_SEQUENCES = 30

# Videos are going to be 30 frames in length
SEQUENCE_LENGTH = 30

In [ ]:
for action in ACTIONS: 
    for sequence in range(N_SEQUENCES):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

## 4. Generate training data

In [ ]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_facemesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5, static_image_mode=True, max_num_faces=1, refine_landmarks=False,) as face:
    # NEW LOOP
    # Loop through actions
    for action in ACTIONS:
        # Loop through sequences aka videos
        for sequence in range(N_SEQUENCES):
            # Loop through video length aka sequence length
            for frame_num in range(SEQUENCE_LENGTH):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, face)
#                 print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'PREPARE FOR RECORDING', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'RECORDING NOW', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

## 5. Visualize data to check if there is anything odd

In [ ]:
plt.rcParams["figure.figsize"] = [7.50, 3.50]
plt.rcParams["figure.autolayout"] = True
for action in ACTIONS:
    plt.title(f'{action} graph')
    for sequence in range(N_SEQUENCES):
        window = []
        for frame_num in range(SEQUENCE_LENGTH):
            try:
                res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
                window.append(res)
            except:
                pass
        #visualize window
        nose_x = [r[0] for r in window]
        nose_y = [r[1] for r in window]
        nose_z = [r[2] for r in window]
        colorX = "green" if action == "naying" else "red"
        colorY = "green" if action == "nodding" else "red"
        plt.plot(nose_x, color=colorX)
        plt.plot(nose_y, color=colorY)
        #plt.plot(nose_z, color="gray")
    plt.show()

## 6. Preprocess Data and Create Features and Labels

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label:num for num, label in enumerate(ACTIONS)}

In [ ]:
sequences, labels = [], []
for action in ACTIONS:
    for sequence in range(N_SEQUENCES):
        if not os.path.isdir(os.path.join(DATA_PATH, action, str(sequence))): #in case we manually deleted some recordings
            continue
        window = []
        for frame_num in range(SEQUENCE_LENGTH):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
len(res)

In [ ]:
X = np.array(sequences)

In [ ]:
X.shape

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

## 7. Build and Train LSTM Neural Network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=False, activation='relu', input_shape=(30,9)))
#model.add(LSTM(128, return_sequences=True, activation='relu'))
#model.add(LSTM(64, return_sequences=False, activation='relu'))
#model.add(Dense(64, activation='relu'))
#model.add(Dense(32, activation='relu'))
model.add(Dense(ACTIONS.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy']) 

In [ ]:
#Wichtig! Use binary_crossentropy statt categorical_crossentropy wenn ich nur 2 Labels hab (nodding & nicht -nodding)!!

In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

In [ ]:
model.summary()

## 8. Make Predictions

In [ ]:
res

In [ ]:
model.predict(X_test)

In [ ]:
ACTIONS[np.argmax(res[2])]

In [ ]:
ACTIONS[np.argmax(y_test[2])]

## 9. Save Weights

In [ ]:
model.save('action.h5')

## 10. Evaluation using Confusion Matrix and Accuracy

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [ ]:
yhat

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

## (Optional) Load weights into model

In [ ]:
model.load_weights('action.h5')

## 11. Test in Real Time

In [ ]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))


In [ ]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_facemesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5, static_image_mode=True, max_num_faces=1, refine_landmarks=False,) as face:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, face)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(ACTIONS[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if ACTIONS[np.argmax(res)] != sentence[-1]:
                        sentence.append(ACTIONS[np.argmax(res)])
                else:
                    sentence.append(ACTIONS[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, ACTIONS, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()